In [130]:
import torch
from d2l import torch as d2l
from IPython import display

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(
    batch_size)  #获得分类数据集,其中结果已经编码化,并且已经为小批量的数据集


In [131]:
num_inputs=784
num_outputs=10
w=torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b=torch.zeros(num_outputs,requires_grad=True)

In [132]:
x=torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.]])
x.sum(0,keepdim=True),x.sum(1,keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [133]:
def softmax(x):#x是一个向量n*d
    x_exp=x.exp()
    partition=x_exp.sum(1,keepdim=True)#n*1
    return x_exp/partition

In [134]:
x=torch.normal(0,1,(2,5))
x_prob=softmax(x)
x_prob,x_prob.sum()

(tensor([[0.3487, 0.0526, 0.0809, 0.1685, 0.3493],
         [0.4110, 0.2032, 0.0398, 0.2479, 0.0980]]),
 tensor(2.))

In [135]:
def net(x):
    return softmax(torch.matmul(x.reshape(-1,w.shape[0]),w)+b)


In [136]:
y=torch.tensor([0,2])#torch.Tensor是表示torch.FloatTensor
y_hat=torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],y]

tensor([0.1000, 0.5000])

In [137]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[:,y])

cross_entropy(y_hat,y)

tensor([[2.3026, 0.5108],
        [1.2040, 0.6931]])

In [138]:
def accuracy(y_hat,y):#y_hat是预测的结果,y是已经编码的结果
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y
    return float(cmp.type(y.dtype).sum())

In [139]:
accuracy(y_hat,y)/len(y)

0.5

In [140]:
from tqdm import tqdm
def evaluate_accuracy(net,data_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()
    a=Accumulator(2)
    for X,y in tqdm(data_iter):
        a.add(accuracy(net(X),y),y.numel())#tensor.numel()返回tensor的元素个数
    return a[0]/a[1]
    

In [141]:
class Accumulator():
    def __init__(self,num):
        self.data=[0.0]*num
    def __getitem__(self,ind):
        return self.data[ind]
    def add(self,*argv):
        self.data=[a+float(b) for a,b in zip(self.data,argv)]
    def clear(self):
        self.data=[0.0]*len(self.data)

In [142]:
evaluate_accuracy(net,data_iter=test_iter)

100%|██████████| 40/40 [00:02<00:00, 18.17it/s]


0.1309